In [1]:
import pandas as pd
import numpy as np

class Annual_subfactors(object):
    # input only fiscal quarter 4 data for df
    def __init__(self, df, dfcpi):

        self.df = df
        self.dfcpi = dfcpi
        self.df.index = self.df.fyear
        self.df = self.df.join(self.dfcpi.cpi)
        self.df.index = self.df.pdate
        # fiscal year
        self.fyear = self.df['fyear']
        #self.tic = self.df['tic']
        # total assets
        self.at = self.df['at']
        # total liabilities
        self.lt = self.df['lt']
        # stockholders equity
        self.seq = self.df['seq']
        #
        self.txdi = self.df['txdi']
        # book value of preferred stock at par value
        self.pstk = self.df['pstk']
        # income before extraordinary items
        self.ib = self.df['ib']
        # balance sheet deferred taxes and investment tax credit
        self.txditc = self.df['txditc']
        # dividends on common stock
        self.dvc = self.df['dvc']
        # purchase of common and preferred stock
        self.prstkc = self.df['prstkc']
        # preferred stock outstanding / redeemable
        self.pstkrv = self.df['pstkrv']
        # preferred
        self.preferred = self.df['preferred']
        # sale of common and preferred stock
        self.sstk = self.df['sstk']
        # cash and short term investments
        self.che = self.df['che']
        # debt incl. in current liabilities
        self.dlc = self.df['dlc']
        # long term debt
        self.dltt = self.df['dltt']
        # minority interests
        self.mib = self.df['mib']
        # common equity
        self.ceq = self.df['ceq']
        # gross property plant and equipment
        self.ppegt = self.df['ppegt']
        # inventory
        self.invt = self.df['invt']
        # capital expenditure
        self.capx = self.df['capx']
        # equitys share of depriciation
        self.dp = self.df['dp']
        # net income
        self.ni = self.df['ni']
        # Operating Activities Net Cash Flow  - Statement of Cash Flows
        self.oancf = self.df['oancf']
        # cash dividends - cash flow statement
        self.dv = self.df['dv']
        # Investing Activities/Net Cash Flow
        self.ivncf = self.df['ivncf']
        # Financing Activities/Net Cash Flow
        self.fincf =self.df['fincf']
        #
        self.ivao = self.df['ivao']
        self.ivst = self.df['ivst']
        # sales(net) or revenue
        self.sale = self.df['sale']
        # cost of goods sold
        self.cogs = self.df['cogs']
        # selling, general, and administrative expenses
        self.xsga = self.df['xsga']
        # adjustment factor
        self.ajex = self.df['ajex']
        # shares outstanding
        self.csho = self.df['csho']
        # current assets
        self.act = self.df['act']
        # current liabilities
        self.lct = self.df['lct']
        # taxed payable
        self.txp = self.df['txp']

        self.book_equity = self.seq + self.txditc - self.preferred


    def o_no(self, net=False):
        #sprint('o_no')
        issuances = self.sstk- np.maximum(
            self.pstkrv - self.pstkrv.shift(1), 0)
        # todo: check if to use -ve value or +ve value of change
        repurchases = self.prstkc + np.minimum(
            self.pstkrv - self.pstkrv.shift(1), 0)
        o = self.dvc + repurchases
        no = o - issuances
        op = o
        nop = no

        if net == False:
            return op
        else:
            return nop

    def i_a(self):
        #print('i_a')
        factor = (self.at / self.at.shift(1)) - 1
        return factor

    def noa(self):
        #print('noa')
        oa = self.at - self.che
        ol = self.at - self.dlc - self.dltt - self.mib - self.pstk - self.ceq
        noa = oa - ol
        #  lag for assets
        scaled = noa / self.at.shift(1)
        return scaled

    def pia(self):
        #print('pia')
        chg_ppe = self.ppegt - self.ppegt.shift(1)
        chg_inv = self.invt - self.invt.shift(1)
        pia = (chg_ppe + chg_inv) / self.at.shift(1)

        return pia

    def ig(self):
        #print('ig')
        g = (self.capx / self.capx.shift(1)) - 1
        # todo: clean capx
        #g[np.isinf(g)] = np.nan
        return g

    def nsi(self):
        #print('nsi')
        split = self.csho * self.ajex
        lnsi = np.log(split / split.shift(1))
        return lnsi

    def ivg(self):
        #print('ivg')
        g = (self.invt / self.invt.shift(1)) - 1
        return g

    def ivc(self):
        #print('ivc')
        chg = self.invt - self.invt.shift(1)
        avg = (self.at + self.at.shift(1)) / 2
        factor = chg / avg
        return factor

    def oa(self):
        #print('oa')
        d_ca = self.act - self.act.shift(1)
        d_cash = self.che - self.che.shift(1)
        d_cl = self.lct - self.lct.shift(1)
        d_std = self.dlc - self.dlc.shift(1)
        d_tp = self.txp - self.txp.shift(1)

        sloan = (d_ca-d_cash)-(d_cl-d_std-d_tp)-self.dp
        hirbar = self.ni - self.oancf

        factor = np.where(self.fyear < 1988, sloan, hirbar) / self.at.shift(1)
        return factor

    def poa(self):
        #print('poa')
        d_ca = self.act - self.act.shift(1)
        d_cash = self.che - self.che.shift(1)
        d_cl = self.lct - self.lct.shift(1)
        d_std = self.dlc - self.dlc.shift(1)
        d_tp = self.txp - self.txp.shift(1)

        sloan = (d_ca - d_cash) - (d_cl - d_std - d_tp) - self.dp
        hirbar = self.ni - self.oancf
        scaled = np.abs(self.ni)
        opac = np.where(self.fyear < 1988, sloan, hirbar)

        factor = opac/scaled
        return factor

    def pta(self):
        #print('pta')
        nncwc = (self.act - self.che) - (self.lct - self.dlc)
        d_wc = nncwc - nncwc.shift(1)
        nncoa = (self.at - self.act - self.ivao) - (self.lt - self.lct - self.dltt)
        d_nco = nncoa - nncoa.shift(1)
        nfa = (self.ivst + self.ivao) - (self.dltt + self.dlc + self.pstk)
        d_fin = nfa - nfa.shift(1)

        sloan = d_wc + d_nco + d_fin
        hirbar = (self.ni - (self.oancf + self.ivncf + self.fincf) + self.sstk - (self.prstkc + self.dv))

        tpac = np.where(self.fyear < 1988, sloan, hirbar)
        scaled = np.abs(self.ni)
        factor = tpac/scaled
        return factor

    def gpa(self):
        #print('gpa')
        gp = self.sale - self.cogs
        factor = gp / self.at
        return factor

    def oc_a(self):
        #print('oc_a')
        g = 0.1
        delta = 0.15
        dfn = self.df[['xsga', 'cpi']]
        dfn['oc'] = np.nan

        dfn.ix[0,'oc'] = dfn.ix[0,'xsga'] / (g+delta)
        for i in range(1, len(dfn)):
            dfn.ix[i, 'oc'] = (1-delta)*dfn.ix[i-1, 'oc'] + dfn.ix[i, 'xsga']/dfn.ix[i, 'cpi']

        factor = dfn.oc / self.at
        return factor

    def ol(self):
        #print('ol')
        oc = self.cogs + self.xsga
        factor = oc / self.at

        return factor


    def output(self, list_factors='ALL'):
        if list_factors == 'ALL':
            list_factors = [self.book_equity, self.ib, self.dp, self.at, self.txdi, self.preferred, self.o_no(), self.o_no(net=True), self.i_a(),
                            self.noa(), self.pia(), self.ig(), self.nsi(), self.ivg(), self.ivc(), self.oa(), self.poa(),
                            self.pta(), self.gpa(), self.oc_a(), self.ol()]

        dff = pd.concat([factor for factor in list_factors], axis=1)
        dff.insert(0, column='gvkey', value=self.df.gvkey)
        #dff.insert(1, column='tic', value=self.df.tic)
        dff.insert(1, column='cusip', value=self.df.cusip)
        dff.columns = ['gvkey', 'cusip', 'book_equity', 'ib', 'dp', 'at', 'txdi', 'preferred', 'o', 'no', 'i_a',
                       'noa', 'pia', 'ig', 'nsi', 'ivg', 'ivc', 'oa', 'poa', 'pta', 'gpa', 'oc_a', 'ol']
        dff.index.names = ['pdate']

        return dff


class Annual_factors(object):
    # merged data
    def __init__(self, df):

        self.df = df
        self.df.index = self.df.date
        self.gvkey = self.df['gvkey']
        # ticker
        self.tic = self.df['tic']
        #
        self.cusip = self.df['cusip']
        #self.permno = self.df['PERMNO']
        # book equity
        self.book_equity = self.df['book_equity']
        # market equity
        self.equity = self.df['equity']
        # extraordinary items
        self.ib = self.df['ib']
        # depreciation
        self.dp = self.df['dp']
        # total asset
        self.at = self.df['at']
        # deferred taxes
        self.txdi = self.df['txdi']
        # total payout
        self.o_ = self.df['o']
        # net payout
        self.no_ = self.df['no']
        # investment to asset
        self.i_a = self.df['i_a']
        # net operating assets
        self.noa = self.df['noa']
        # pia
        self.pia = self.df['pia']
        # investment growth
        self.ig = self.df['ig']
        # net stock issue
        self.nsi = self.df['nsi']
        # inventory growth
        self.ivg = self.df['ivg']
        # inventory change
        self.ivc = self.df['ivc']
        # operating accurals
        self.oa = self.df['oa']
        # percent accurals
        self.poa = self.df['poa']
        # pta
        self.pta = self.df['pta']
        # gross profit to assets
        self.gpa = self.df['gpa']
        # organization capital
        self.oc_a = self.df['oc_a']
        # operating leverage
        self.ol = self.df['ol']

    # book to market
    def b_m(self):
        value = self.book_equity / self.equity
        return value

    # earnings to price
    def e_p(self):
        value = self.ib / self.equity
        return value

    # cashflow to price
    def cf_p(self):
        value = (self.ib + ((self.equity / (self.at - self.book_equity + self.equity)) * self.dp) + self.txdi)/self.equity
        return value

    def o(self):
        factor = self.o_ / self.equity
        return factor

    def no(self):
        factor = self.no_ / self.equity
        return factor


    def output(self, list_factors='ALL'):
        if list_factors == 'ALL':
            list_factors = [self.b_m(), self.e_p(), self.cf_p(), self.o(), self.no(), self.i_a,
                            self.noa, self.pia, self.ig, self.nsi, self.ivg, self.ivc, self.oa,
                            self.poa, self.pta, self.gpa, self.oc_a, self.ol, self.df.R6, self.df.R11, self.df.PERMNO,
                            self.equity, self.df.adj_price, self.df.forward_r, self.df.CUSIP, self.df.PRC, self.df.SHROUT,
                            self.df.tr0, self.df.tr1, self.df.SPREAD, self.df.CFACPR, self.df.adj_SPREAD, self.df.forward_SPREAD]

        dff = pd.concat([factor for factor in list_factors], axis=1)
        #dff.insert(0, column='gvkey', value=self.df.gvkey)
        dff.insert(0, column='tic', value=self.df.tic)
        #dff.insert(2, column='PERMNO', value=self.df.permno)
        dff.columns = ['tic','b_m', 'e_p', 'cf_p', 'o', 'no', 'i_a',
                       'noa', 'pia', 'ig', 'nsi', 'ivg', 'ivc', 'oa', 'poa',
                       'pta', 'gpa', 'oc_a', 'ol', 'R6', 'R11', 'PERMNO', 'equity', 'adj_price', 'forward_r', 'CUSIP', 'PRC', 'SHROUT',
                       'tr0', 'tr1', 'SPREAD', 'CFACPR', 'adj_SPREAD', 'forward_SPREAD']
        dff.index.names = ['date']

        return dff


In [10]:
class Quarterly_subfactors(object):
    def __init__(self, df):
        self.df = df
        #self.gvkey = gvkey
        self.df.index = self.df.pdate
        self.rdq = self.df['rdq']
        self.eps = self.df['epspiq']
        self.ibq = self.df['ibq']
        self.atq = self.df['atq']
        self.ltq = self.df['ltq']
        self.seqq = self.df['seqq']
        self.txditcq = self.df['txditcq']
        self.pstk = self.df['pstkrq']
        self.preferred = self.df['preferred']
        self.book_equity = self.seqq + self.txditcq - self.preferred


    def sue(self):
        # lag difference; lag = 4
        chg_eps = self.eps.diff(4)
        # std of change in eps over 8 periods
        std = chg_eps.rolling(window=8, min_periods=6).std()
        sue = chg_eps / std
        return sue

    def roe(self):
        factor = self.ibq / self.book_equity.shift(1)

        return factor

    def roa(self):
        factor = self.ibq / self.atq.shift(1)

        return factor

    def nei(self):
        # 1 if ibq increased over the same quarter a year ago; else 0
        y = np.where(self.ibq.diff(4) > 0, 1, 0)
        # convert to pandas Series
        y = pd.Series(y, index=self.ibq.index)
        # return nei
        sum = y * (y.groupby((y != y.shift()).cumsum()).cumcount() + 1)

        return sum

    def output(self, list_factors='ALL'):
        if list_factors == 'ALL':
            list_factors = [self.sue(), self.roe(), self.roa(), self.nei()]

        dff = pd.concat([factor for factor in list_factors], axis=1)
        #dff.insert(0, column='gvkey', value=self.df.gvkey)
        dff.insert(0, column='tic', value=self.df.tic)
        dff.insert(1, column='cusip', value=self.df.cusip)
        dff.columns = ['tic', 'cusip', 'sue', 'roe', 'roa', 'nei']
        dff.index.names = ['pdate']

        return dff

In [2]:
comp_q = pd.read_csv('/Users/guapofish/Desktop/Factor Investing/df_Q_output.csv')
comp_a = pd.read_csv('/Users/guapofish/Desktop/Factor Investing/df_A_output.csv')


In [3]:
# price data
# get the link table from CRSP 
link = pd.read_csv('/Users/guapofish/Desktop/Factor Investing/link_table.csv')
link.head(10)


,gvkey,conm,tic,cusip,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT
0,1000,A & E PLASTIK PAK INC,AE.2,000032102,P,01,LU,25881,23369,19701113,19780630
1,1001,A & M FOOD SERVICES INC,AMFD.,000165100,P,01,LU,10015,6398,19830920,19860731
2,1002,AAI CORP,AAIC,000352104,C,01,LC,10023,22159,19721214,19730605
3,1003,A.A. IMPORTING CO INC,ANTQ,000354100,C,01,LU,10031,6672,19831207,19890816
4,1004,AAR CORP,AIR,000361105,P,01,LU,54594,20000,19720424,E
5,1005,A.B.A. INDUSTRIES INC,ABA.2,000370106,C,01,LU,61903,11,19730131,19830131
6,1007,ABKCO INDUSTRIES INC,4135B,000774109,C,00X,LU,10058,20,19731001,19790130
7,1007,ABKCO INDUSTRIES INC,4135B,000774109,P,01,LU,10058,20,19790131,19840928
8,1008,ABM COMPUTER SYSTEMS INC,ABMC.,000775106,P,01,LC,10066,6331,19830825,19871030
9,1009,ABS INDUSTRIES INC,ABSI,000781104,C,01,LC,10074,5271,19820118,19960313


In [4]:
# for the stocks in our annual fundamentals aka compustat file, we identify the PERMNO codes using the link table
link = link[link.gvkey.isin(comp_a.gvkey.unique())]
link2 = link[link.cusip.isin(comp_a.cusip.unique())]
link2.head(10)

,gvkey,conm,tic,cusip,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT
0,1000,A & E PLASTIK PAK INC,AE.2,000032102,P,01,LU,25881,23369,19701113,19780630
1,1001,A & M FOOD SERVICES INC,AMFD.,000165100,P,01,LU,10015,6398,19830920,19860731
4,1004,AAR CORP,AIR,000361105,P,01,LU,54594,20000,19720424,E
5,1005,A.B.A. INDUSTRIES INC,ABA.2,000370106,C,01,LU,61903,11,19730131,19830131
12,1011,ACS ENTERPRISES INC,ACSE,000872309,P,01,LC,10082,5793,19830321,19950928
14,1013,ADC TELECOMMUNICATIONS INC,ADCT,000886309,P,01,LU,50906,2902,19790316,20101231
16,1016,AEC INC,AECE,001015106,P,01,LC,10146,314,19771130,19880426
17,1017,AEL INDUSTRIES -CL A,AELNA,001030105,P,01,LC,10154,81,19721214,19960229
25,1023,AGS COMPUTERS INC,AGS.2,001240100,P,01,LU,66755,5542,19820609,19881005
29,1027,A I M COS,AIM.2,001412105,P,01,LU,28601,23555,19660131,19710528


In [5]:
# get the list of unique PERMNO numbers
pd.DataFrame(link2.LPERMNO.unique()).to_csv('PERMNO.txt', index=False, header=False)


In [6]:
link2['PERMNO'] = link2.LPERMNO
 #get the monthly price for the PERMNO s from CRSP from 1961-01 to latest
raw_monthly_prices_file = '/Users/guapofish/Desktop/Factor Investing/wprice_0227.csv'
p = pd.read_csv(raw_monthly_prices_file)
p

/Users/guapofish/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/guapofish/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5,6,9,18,19,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,PERMNO,date,NAMEENDT,SHRCD,EXCHCD,SICCD,NCUSIP,TICKER,COMNAM,SHRCLS,...,CFACSHR,ALTPRC,SPREAD,ALTPRCDT,RETX,vwretd,vwretx,ewretd,ewretx,sprtrn
0,10008,1985/12/31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-14.06250,NaN,1986/01/16,NaN,0.043061,0.040080,0.028021,0.026355,0.045061
1,10008,1986/01/31,1988/11/21,10.0,3.0,3430,36547310,GACO,GARDENAMERICA CORP,NaN,...,1.0,-14.62500,0.25,1986/01/31,C,0.009829,0.008006,0.044071,0.043082,0.002367
2,10008,1986/02/28,NaN,10.0,3.0,3430,36547310,GACO,GARDENAMERICA CORP,NaN,...,1.0,-16.37500,0.75,1986/02/28,0.119658,0.072500,0.068190,0.060381,0.058938,0.071489
3,10008,1986/03/31,NaN,10.0,3.0,3430,36547310,GACO,GARDENAMERICA CORP,NaN,...,1.0,-17.75000,0.50,1986/03/31,0.083969,0.053885,0.051360,0.047192,0.045679,0.052794
4,10008,1986/04/30,NaN,10.0,3.0,3430,36547310,GACO,GARDENAMERICA CORP,NaN,...,1.0,-18.25000,0.50,1986/04/30,0.028169,-0.007903,-0.009633,0.016140,0.015141,-0.014148
5,10008,1986/05/30,NaN,10.0,3.0,3430,36547310,GACO,GARDENAMERICA CORP,NaN,...,1.0,-18.00000,0.50,1986/05/30,-0.013699,0.050844,0.047123,0.036194,0.034779,0.050229
6,10008,1986/06/30,NaN,10.0,3.0,3430,36547310,GACO,GARDENAMERICA CORP,NaN,...,1.0,-17.12500,0.75,1986/06/30,-0.048611,0.014246,0.011624,0.008087,0.006715,0.014110
7,10008,1986/07/31,NaN,10.0,3.0,3430,36547310,GACO,GARDENAMERICA CORP,NaN,...,1.0,-14.50000,0.50,1986/07/31,-0.153285,-0.059700,-0.061454,-0.073128,-0.073917,-0.058683
8,10008,1986/08/29,NaN,10.0,3.0,3430,36547310,GACO,GARDENAMERICA CORP,NaN,...,1.0,-13.62500,0.25,1986/08/29,-0.060345,0.066181,0.062428,0.023269,0.021890,0.071193
9,10008,1986/09/30,NaN,10.0,3.0,3430,36547310,GACO,GARDENAMERICA CORP,NaN,...,1.0,11.50000,NaN,1986/09/30,-0.155963,-0.079021,-0.081274,-0.059071,-0.060184,-0.085439


In [9]:
# edit the price data and link the gvkeys
link2['COMNAM'] = link2.conm
link = link2[['gvkey', 'tic', 'cusip', 'PERMNO', 'COMNAM']]
link = link.drop_duplicates(subset=['gvkey', 'cusip'])
link.index = link.COMNAM

p = p.join(link, on=['COMNAM'], rsuffix= '_link')
p.to_csv('linkedprice.csv', index=False)


In [11]:
dfq = comp_q.groupby('gvkey').apply(lambda x: Quarterly_subfactors(x).output())
dfq.to_csv('/Users/guapofish/Desktop/Factor Investing/Q_subfactors.csv')


In [12]:
# from now all group by gvkey

dfq = pd.read_csv('/Users/guapofish/Desktop/Factor Investing/Q_subfactors.csv')
# at times release dates of two quarters are similar. We use the latest
dfq = dfq.drop_duplicates(subset=['gvkey', 'pdate'], keep='last')


In [13]:
dfq.head(20)


,gvkey,pdate,tic,cusip,sue,roe,roa,nei
1,1000,1972-04-30,AE.2,32102,NaN,0.040568,0.011592,0
2,1000,1972-07-31,AE.2,32102,NaN,0.054272,0.016979,0
3,1000,1972-10-31,AE.2,32102,NaN,0.062895,0.014252,0
4,1000,1973-02-28,AE.2,32102,NaN,0.041527,0.010160,1
5,1000,1973-04-30,AE.2,32102,NaN,0.051170,0.018787,2
6,1000,1973-07-31,AE.2,32102,NaN,0.076953,0.029437,3
7,1000,1973-10-31,AE.2,32102,NaN,0.023256,0.009394,0
8,1000,1974-02-28,AE.2,32102,NaN,0.087745,0.035967,1
9,1000,1974-04-30,AE.2,32102,0.068589,0.037622,0.015204,0
10,1000,1974-07-31,AE.2,32102,0.338104,0.079413,0.031096,1


In [14]:
# merge subfactors with prices
# price data from CRSP that has been linked to gvkey and cusip
pdf = pd.read_csv('/Users/guapofish/Desktop/Factor Investing/linkedprice.csv')

/Users/guapofish/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5,6,9,18,19,24,66,71) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
pdf.head(10)


,PERMNO,date,NAMEENDT,SHRCD,EXCHCD,SICCD,NCUSIP,TICKER,COMNAM,SHRCLS,...,gvkey,tic,cusip,PERMNO_link,COMNAM_link,gvkey_link,tic_link,cusip_link,PERMNO_link.1,COMNAM_link.1
0,10008,1985/12/31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10008,1986/01/31,1988/11/21,10.0,3.0,3430,36547310,GACO,GARDENAMERICA CORP,NaN,...,12165.0,GACO.,365473107,10008.0,GARDENAMERICA CORP,12165.0,GACO.,365473107,10008.0,GARDENAMERICA CORP
2,10008,1986/02/28,NaN,10.0,3.0,3430,36547310,GACO,GARDENAMERICA CORP,NaN,...,12165.0,GACO.,365473107,10008.0,GARDENAMERICA CORP,12165.0,GACO.,365473107,10008.0,GARDENAMERICA CORP
3,10008,1986/03/31,NaN,10.0,3.0,3430,36547310,GACO,GARDENAMERICA CORP,NaN,...,12165.0,GACO.,365473107,10008.0,GARDENAMERICA CORP,12165.0,GACO.,365473107,10008.0,GARDENAMERICA CORP
4,10008,1986/04/30,NaN,10.0,3.0,3430,36547310,GACO,GARDENAMERICA CORP,NaN,...,12165.0,GACO.,365473107,10008.0,GARDENAMERICA CORP,12165.0,GACO.,365473107,10008.0,GARDENAMERICA CORP
5,10008,1986/05/30,NaN,10.0,3.0,3430,36547310,GACO,GARDENAMERICA CORP,NaN,...,12165.0,GACO.,365473107,10008.0,GARDENAMERICA CORP,12165.0,GACO.,365473107,10008.0,GARDENAMERICA CORP
6,10008,1986/06/30,NaN,10.0,3.0,3430,36547310,GACO,GARDENAMERICA CORP,NaN,...,12165.0,GACO.,365473107,10008.0,GARDENAMERICA CORP,12165.0,GACO.,365473107,10008.0,GARDENAMERICA CORP
7,10008,1986/07/31,NaN,10.0,3.0,3430,36547310,GACO,GARDENAMERICA CORP,NaN,...,12165.0,GACO.,365473107,10008.0,GARDENAMERICA CORP,12165.0,GACO.,365473107,10008.0,GARDENAMERICA CORP
8,10008,1986/08/29,NaN,10.0,3.0,3430,36547310,GACO,GARDENAMERICA CORP,NaN,...,12165.0,GACO.,365473107,10008.0,GARDENAMERICA CORP,12165.0,GACO.,365473107,10008.0,GARDENAMERICA CORP
9,10008,1986/09/30,NaN,10.0,3.0,3430,36547310,GACO,GARDENAMERICA CORP,NaN,...,12165.0,GACO.,365473107,10008.0,GARDENAMERICA CORP,12165.0,GACO.,365473107,10008.0,GARDENAMERICA CORP


In [16]:
# intersection of tickers
inte = list(set.intersection(set(dfq.tic.unique()), set(pdf.tic.unique())))

dfq = dfq[dfq.tic.isin(inte)]
pdf = pdf[pdf.tic.isin(inte)]

pdf = pdf.drop_duplicates(subset=['date', 'gvkey'])


In [17]:
def stringp(df):
    df = df[:4]+'/'+df[5:7]+'/'+df[8:]
    return df

dfq.pdate = dfq.pdate.apply(stringp)


In [18]:
dfq.index = pd.MultiIndex.from_arrays([dfq.gvkey,dfq.pdate])


In [19]:
pdf2 = pdf.join(dfq, on=['gvkey', 'date'], rsuffix='_c')


In [20]:
# equity is market equity = price * common shares outstanding / 1000
# number of shares are in thousands while everything else in fundamental is in millions
pdf2['equity'] = pdf2.PRC.abs()*pdf.SHROUT/1000.0

# prices adjusted to splits etc.
pdf2['adj_price'] = pdf2.PRC.abs()/pdf.CFACPR

# RET when missing try DLRET
pdf2['RET'] = np.where(pdf2.RET.isna(), pdf2.DLRET, pdf2.RET)

# SPREAD
pdf2['SPREAD'] = np.where(pdf2.SPREAD.isna(), pdf2.ASK-pdf2.BID, pdf2.SPREAD)
pdf2['adj_SPREAD'] = pdf2.SPREAD/pdf2.CFACPR
#pdf2['adj_SPREAD'] = pdf2.adj_SPREAD.fillna(0)

In [21]:
# merge the quarterly file into a monthly price series
# slice dataframe for each stock from first available fundamental data timestamp till whenever the prices are available
def merge(df):
    ind_c = df['pdate'].first_valid_index()
    df = df.loc[ind_c: ]
    df['forward_r'] = df.RET.shift(-1)
    df['forward_SPREAD'] = df.adj_SPREAD.shift(-1)
    return df


In [22]:
pdf2 = pdf2.groupby('gvkey').apply(merge)

pdf2['tr0'] = pdf2.adj_SPREAD/pdf2.adj_price
pdf2['tr1'] = pdf2.forward_SPREAD/pdf2.adj_price

In [23]:
finalQ = pdf2[['date','PERMNO', 'PERMCO', 'CUSIP', 'PRC', 'SHROUT', 'tic', 'sue', 'roe', 'roa', 'nei', 'equity',
               'adj_price', 'forward_r', 'tr0', 'tr1', 'SPREAD', 'CFACPR', 'adj_SPREAD', 'forward_SPREAD']]


In [24]:
finalQ


date  PERMNO  PERMCO     CUSIP      PRC   SHROUT   tic  \
gvkey                                                                           
1000.0   366896  1972/07/31   25881   23369  00003210   6.0000   2655.0  AE.2   
         366897  1972/08/31   25881   23369  00003210   5.8750   2655.0  AE.2   
         366898  1972/09/29   25881   23369  00003210   5.2500   2655.0  AE.2   
         366899  1972/10/31   25881   23369  00003210   5.6250   2655.0  AE.2   
         366900  1972/11/30   25881   23369  00003210   5.5000   2655.0  AE.2   
         366901  1972/12/29   25881   23369  00003210   5.1250   2655.0  AE.2   
         366902  1973/01/31   25881   23369  00003210   5.1250   2655.0  AE.2   
         366903  1973/02/28   25881   23369  00003210   4.2500   2655.0  AE.2   
         366904  1973/03/30   25881   23369  00003210   4.3750   2655.0  AE.2   
         366905  1973/04/30   25881   23369  00003210   3.3750   2655.0  AE.2   
         366906  1973/05/31   25881   23369  00003210   3.1250   2655.0  AE.2   
         366907  1973/06/29   25881   23369  00003210   3.0000   2655.0  AE.2   
         366908  1973/07/31   25881   23369  00003210   3.8750   2655.0  AE.2   
         366909  1973/08/31   25881   23369  00003210   2.8750   2655.0  AE.2   
         366910  1973/09/28   25881   23369  00003210   3.0000   2655.0  AE.2   
         366911  1973/10/31   25881   23369  00003210   2.5000   2655.0  AE.2   
         366912  1973/11/30   25881   23369  00003210   1.7500   2655.0  AE.2   
         366913  1973/12/31   25881   23369  00003210   1.7500   2655.0  AE.2   
         366914  1974/01/31   25881   23369  00003210   2.2500   2655.0  AE.2   
         366915  1974/02/28   25881   23369  00003210  -2.5625   2655.0  AE.2   
         366916  1974/03/29   25881   23369  00003210   2.6250   2655.0  AE.2   
         366917  1974/04/30   25881   23369  00003210   2.8750   2655.0  AE.2   
         366918  1974/05/31   25881   23369  00003210   2.5000   2655.0  AE.2   
         366919  1974/06/28   25881   23369  00003210   2.6250   2655.0  AE.2   
         366920  1974/07/31   25881   23369  00003210   2.2500   2655.0  AE.2   
         366921  1974/08/30   25881   23369  00003210   2.6250   2655.0  AE.2   
         366922  1974/09/30   25881   23369  00003210   2.1250   2655.0  AE.2   
         366923  1974/10/31   25881   23369  00003210   2.2500   2655.0  AE.2   
         366924  1974/11/29   25881   23369  00003210   2.2500   2217.0  AE.2   
         366925  1974/12/31   25881   23369  00003210   2.1250   2217.0  AE.2   
...                     ...     ...     ...       ...      ...      ...   ...   
296866.0 216737  2018/06/29   16179   55622  62844N10   0.9115  29679.0  MYSZ   
         216738  2018/07/31   16179   55622  62844N10   0.7500  29679.0  MYSZ   
         216739  2018/08/31   16179   55622  62844N10   0.8655  29710.0  MYSZ   
         216740  2018/09/28   16179   55622  62844N10   1.1700  29710.0  MYSZ   
         216741  2018/10/31   16179   55622  62844N10   0.9890  29710.0  MYSZ   
         216742  2018/11/30   16179   55622  62844N10   1.0400  29852.0  MYSZ   
         216743  2018/12/31   16179   55622  62844N10   0.7704  29852.0  MYSZ   
311524.0 152849  2013/04/30   13861   54367  87509U10  15.1200  65228.0   TAM   
         152850  2013/05/31   13861   54367  87509U10  18.8500  65228.0   TAM   
         152851  2013/06/28   13861   54367  87509U10  20.3900  65228.0   TAM   
         152852  2013/07/31   13861   54367  87509U10  22.8000  65228.0   TAM   
         152853  2013/08/30   13861   54367  87509U10  19.7500  66399.0   TAM   
         152854  2013/09/30   13861   54367  87509U10  20.3000  66399.0   TAM   
         152855  2013/10/31   13861   54367  87509U10  19.9700  66399.0   TAM   
         152856  2013/11/29   13861   54367  87509U10  21.5200  66411.0   TAM   
         152857  2013/12/31   13861   54367  87509U10  20.2100  66411.0   TAM   
         152858  2014/01/31   13861   54367  87509U10 

In [25]:
# using forward fill we essentially use latest available factor every month
def fill(df):
    df['sue'] = df.sue.fillna(method='ffill')
    df['roe'] = df.roe.fillna(method='ffill')
    df['roa'] = df.roa.fillna(method='ffill')
    df['nei'] = df.nei.fillna(method='ffill')

    return df

finalQ = finalQ.groupby('gvkey').apply(fill)


In [26]:
finalQ.dropna()


date  PERMNO  PERMCO     CUSIP      PRC    SHROUT  \
gvkey                                                                       
1000.0   366929   1975/04/30   25881   23369  00003210  -2.6875    2217.0   
         366930   1975/05/30   25881   23369  00003210  -2.5625    2217.0   
1001.0   510      1986/03/31   10015    6398  00016510  12.2500    3985.0   
         511      1986/04/30   10015    6398  00016510  13.0000    3985.0   
         512      1986/05/30   10015    6398  00016510  14.8750    3985.0   
1043.0   1254880  1997/11/28   80071   30061  62544010  -2.6250    2813.0   
         1254881  1997/12/31   80071   30061  62544010   2.9375    2815.0   
         1254882  1998/01/30   80071   30061  62544010   4.3750    2815.0   
         1254889  1998/08/31   80071   30061  62544010   4.3750    2827.0   
         1254890  1998/09/30   80071   30061  62544010  -5.2500    2827.0   
         1254891  1998/10/30   80071   30061  62544010  -4.3750    2829.0   
         1254892  1998/11/30   80071   30061  62544010   4.1250    2829.0   
         1254893  1998/12/31   80071   30061  62544010   3.0000    2832.0   
         1254894  1999/01/29   80071   30061  62544010   3.0000    2832.0   
         1254895  1999/02/26   80071   30061  62544010   2.8750    2832.0   
         1254899  1999/06/30   80071   30061  62544010   2.1250    2833.0   
         1254900  1999/07/30   80071   30061  62544010   2.1875    2833.0   
1045.0   294998   2014/01/31   21020   20010  02376R10  33.5500  356996.0   
         294999   2014/02/28   21020   20010  02376R10  36.9300  471517.0   
         295000   2014/03/31   21020   20010  02376R10  36.6000  649953.0   
         295001   2014/04/30   21020   20010  02376R10  35.0700  720196.0   
         295002   2014/05/30   21020   20010  02376R10  40.1600  720196.0   
         295003   2014/06/30   21020   20010  02376R10  42.9600  720502.0   
         295004   2014/07/31   21020   20010  02376R10  38.8500  720103.0   
         295005   2014/08/29   21020   20010  02376R10  38.8810  720103.0   
         295006   2014/09/30   21020   20010  02376R10  35.4800  717258.0   
         295007   2014/10/31   21020   20010  02376R10  41.3500  717264.0   
         295008   2014/11/28   21020   20010  02376R10  48.5300  717264.0   
         295009   2014/12/31   21020   20010  02376R10  53.6300  697475.0   
         295010   2015/01/30   21020   20010  02376R10  49.0800  697475.0   
...                      ...     ...     ...       ...      ...       ...   
293754.0 125437   2018/01/31   12667   53713  00835P10   1.9100   59332.0   
         125438   2018/02/28   12667   53713  00835P10   1.9000   59332.0   
         125439   2018/03/29   12667   53713  00835P10   2.0100   59337.0   
         125440   2018/04/30   12667   53713  00835P10   1.5000   59337.0   
         125441   2018/05/31   12667   53713  00835P10   1.6000   59337.0   
         125442   2018/06/29   12667   53713  00835P10   1.1100   59341.0   
         125443   2018/07/31   12667   53713  00835P10   0.9610   59341.0   
         125444   2018/08/31   12667   53713  00835P10   1.2500   59341.0   
         125445   2018/09/28   12667   53713  00835P10   1.2300   64767.0   
         125446   2018/10/31   12667   53713  00835P10   1.2000   64767.0   
         125447   2018/11/30   12667   53713  00835P10   1.0300   64767.0   
296866.0 216724   2017/05/31   16179   55622  62844N10   1.2800   17605.0   
         216725   2017/06/30   16179   55622  62844N10   1.1814   17605.0   
         216726   2017/07/31   16179   55622  62844N10   1.1200   17605.0   
         216727   2017/08/31   16179   55622  62844N10   0.8120   17605.0   
         216728   2017/09/29   16179   55622  62844N10   0.7210   18078.0   
         216729   2017/10/31   16179   55622  62844N10   0.7218   18078.0   
         216730   2017/11/30   16179   55622  62844N10   0.9994   18395.0   
         216731   2017/12/29   16179   55622  62844N10   0.6479   22239.0   
         216732   20

In [27]:
finalQ.to_csv('FinalQ .csv', index=False)


In [39]:
FinalQ = finalQ


In [40]:
FinalQ['date']


gvkey           
1000.0    366896    1972/07/31
          366897    1972/08/31
          366898    1972/09/29
          366899    1972/10/31
          366900    1972/11/30
          366901    1972/12/29
          366902    1973/01/31
          366903    1973/02/28
          366904    1973/03/30
          366905    1973/04/30
          366906    1973/05/31
          366907    1973/06/29
          366908    1973/07/31
          366909    1973/08/31
          366910    1973/09/28
          366911    1973/10/31
          366912    1973/11/30
          366913    1973/12/31
          366914    1974/01/31
          366915    1974/02/28
          366916    1974/03/29
          366917    1974/04/30
          366918    1974/05/31
          366919    1974/06/28
          366920    1974/07/31
          366921    1974/08/30
          366922    1974/09/30
          366923    1974/10/31
          366924    1974/11/29
          366925    1974/12/31
                       ...    
296866.0  216737    20

In [63]:
###
# annual subfactors

# cpi annual (avg) data from US Bureau of Labor Statistics
dfcpi = pd.read_csv('cpi.csv')
dfcpi.columns.values[0] = 'fyear'
dfcpi = dfcpi.copy()
dfcpi.index = dfcpi.fyear


'fyear'

In [64]:
# annual subfactors
#comp_a = pd.read_csv('df_A_output.csv')
dfa = comp_a.groupby('tic').apply(lambda x: Annual_subfactors(x, dfcpi).output())
dfa.to_csv('A_subfactors.csv')

dfa = pd.read_csv('A_subfactors.csv')
pdf = pd.read_csv('linkedprice.csv')

pdf = pdf.drop_duplicates(subset=['date', 'gvkey'])

inte = list(set.intersection(set(dfa.gvkey.unique()), set(pdf.gvkey.unique())))

dfa = dfa[dfa.gvkey.isin(inte)]
pda = pdf[pdf.gvkey.isin(inte)]



C:\Users\zchan\Anaconda3\lib\site-packages\ipykernel_launcher.py:94: RuntimeWarning: invalid value encountered in maximum
C:\Users\zchan\Anaconda3\lib\site-packages\ipykernel_launcher.py:97: RuntimeWarning: invalid value encountered in minimum
C:\Users\zchan\Anaconda3\lib\site-packages\ipykernel_launcher.py:215: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\zchan\Anaconda3\lib\site-packages\ipykernel_launcher.py:217: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\zchan\Anaconda3\lib\site-packages\ipykernel_launcher.py:140: RuntimeWarning: divide by zero encountered in log
C:\Users\zchan\Anaconda3\lib\site-packages\I

In [65]:
def stringp(df):
    df = df[:4]+'/'+df[5:7]+'/'+df[8:]
    return df

dfa.pdate = dfa.pdate.apply(stringp)


In [66]:
# at times release dates of two quarters are similar. We use the latest
dfa = dfa.drop_duplicates(subset=['pdate', 'gvkey'])

dfa.index = pd.MultiIndex.from_arrays([dfa.gvkey, dfa.pdate])

pda2 = pda.join(dfa, on=['gvkey', 'date'], rsuffix='_c')
pda2['equity'] = pda2.PRC.abs()*pda2.SHROUT/1000.0
pda2['adj_price'] = pda2.PRC.abs()/pda2.CFACPR

In [67]:
# RET when missing try DLRET
pda2['RET'] = np.where(pda2.RET.isna(), pda2.DLRET, pda2.RET)


In [68]:
# SPREAD
pda2['SPREAD'] = np.where(pda2.SPREAD.isna(), pda2.ASK-pda2.BID, pda2.SPREAD)
pda2['adj_SPREAD'] = pda2.SPREAD/pda2.CFACPR

In [69]:
def merge(df):
    ind_c = df['pdate'].first_valid_index()
    df = df.loc[ind_c: ]
    df['forward_r'] = df.RET.shift(-1)
    df['forward_SPREAD'] = df.adj_SPREAD.shift(-1)
    df['R6'] = (df.adj_price.shift(1)/df.adj_price.shift(7)) -1
    df['R11'] = (df.adj_price.shift(1) / df.adj_price.shift(12)) - 1
    return df

pda2 = pda2.groupby('gvkey').apply(merge)

pda2['tr0'] = pda2.adj_SPREAD/pda2.adj_price
pda2['tr1'] = pda2.forward_SPREAD/pda2.adj_price


In [70]:
def filla(df):

    subfactor_list = ['book_equity', 'ib', 'dp', 'at', 'txdi', 'preferred', 'o', 'no', 'i_a', 'noa', 'pia', 'ig', 'nsi', 'ivg', 'ivc', 'oa', 'poa', 'pta', 'gpa', 'oc_a', 'ol']
    for i in subfactor_list:
        df[i] = df[i].fillna(method='ffill')

    return df

subA = pda2.groupby('gvkey').apply(filla)

C:\Users\zchan\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: 'gvkey' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  if __name__ == '__main__':


In [71]:
finalA = subA.groupby('gvkey').apply(lambda x: Annual_factors(x).output())
finalA.to_csv('final_A.csv')

C:\Users\zchan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 'gvkey' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.


In [72]:
finalA


tic       b_m       e_p      cf_p         o        no  \
gvkey    date                                                                 
1000.0   1970/12/31  AE.2  0.397137  0.070734  0.110539  0.000000  0.000000   
         1971/01/29  AE.2  0.441264  0.078594  0.121334  0.000000  0.000000   
         1971/02/26  AE.2  0.356978  0.063582  0.100568  0.000000  0.000000   
         1971/03/31  AE.2  0.407320  0.072548  0.113044  0.000000  0.000000   
         1971/04/30  AE.2  0.369430  0.065800  0.103676  0.000000  0.000000   
         1971/05/28  AE.2  0.460449  0.082011  0.125982  0.000000  0.000000   
         1971/06/30  AE.2  0.481379  0.085739  0.131022  0.000000  0.000000   
         1971/07/30  AE.2  0.648388  0.115485  0.170357  0.000000  0.000000   
         1971/08/31  AE.2  0.756452  0.134732  0.195174  0.000000  0.000000   
         1971/09/30  AE.2  0.858676  0.152939  0.218320  0.000000  0.000000   
         1971/10/29  AE.2  0.814641  0.145096  0.208383  0.000000  0.000000   
         1971/11/30  AE.2  0.962758  0.171477  0.241632  0.000000  0.000000   
         1971/12/31  AE.2  0.690674  0.123016  0.180117  0.000000  0.000000   
         1972/01/31  AE.2  0.690674  0.123016  0.180117  0.000000  0.000000   
         1972/02/29  AE.2  0.756452  0.134732  0.195174  0.000000  0.000000   
         1972/03/30  AE.2  0.706022  0.125750  0.183643  0.000000  0.000000   
         1972/04/28  AE.2  0.557386  0.099276  0.149101  0.000000  0.000000   
         1972/05/31  AE.2  0.599453  0.106769  0.158974  0.000000  0.000000   
         1972/06/30  AE.2  0.661896  0.117891  0.173482  0.000000  0.000000   
         1972/07/31  AE.2  0.661896  0.117891  0.173482  0.000000  0.000000   
         1972/08/31  AE.2  0.675979  0.120399  0.176733  0.000000  0.000000   
         1972/09/29  AE.2  0.756452  0.134732  0.195174  0.000000  0.000000   
         1972/10/31  AE.2  0.706022  0.125750  0.183643  0.000000  0.000000   
         1972/11/30  AE.2  0.722068  0.128608  0.187321  0.000000  0.000000   
         1972/12/29  AE.2  0.774902  0.138018  0.199372  0.000000  0.000000   
         1973/01/31  AE.2  0.774902  0.138018  0.199372  0.000000  0.000000   
         1973/02/28  AE.2  0.647746  0.137720  0.213491  0.360430  0.337654   
         1973/03/30  AE.2  0.629239  0.133785  0.208138  0.350132  0.328006   
         1973/04/30  AE.2  0.815680  0.173425  0.261228  0.453875  0.425194   
         1973/05/31  AE.2  0.880934  0.187299  0.279442  0.490185  0.459209   
...                   ...       ...       ...       ...       ...       ...   
296866.0 2018/06/29  MYSZ       NaN       NaN       NaN       NaN       NaN   
         2018/07/31  MYSZ       NaN       NaN       NaN       NaN       NaN   
         2018/08/31  MYSZ       NaN       NaN       NaN       NaN       NaN   
         2018/09/28  MYSZ       NaN       NaN       NaN       NaN       NaN   
         2018/10/31  MYSZ       NaN       NaN       NaN       NaN       NaN   
         2018/11/30  MYSZ       NaN       NaN       NaN       NaN       NaN   
         2018/12/31  MYSZ       NaN       NaN       NaN       NaN       NaN   
311524.0 2013/04/30   TAM  0.527251 -0.083143 -0.063172  0.000000 -0.001014   
         2013/05/31   TAM  0.422920 -0.066691 -0.046329  0.000000 -0.000813   
         2013/06/28   TAM  0.390978 -0.061654 -0.041387  0.000000 -0.000752   
         2013/07/31   TAM  0.349651 -0.055137 -0.035178  0.000000 -0.000672   
         2013/08/30   TAM  0.396529 -0.062530 -0.042238  0.000000 -0.000763   
         2013/09/30   TAM  0.385785 -0.060835 -0.040595  0.000000 -0.000742   
         2013/10/31   TAM  0.392160 -0.061841 -0.041568  0.000000 -0.000754   
         2013/11/29   TAM  0.363849 -0.057376 -0.037286  0.000000 -0.000700   
         2013/12/31   TAM  0.387433 -0.061095 -0.040846  0.000000 -0.000745   
         2014/01/31   TAM  0.389360 -0.061399 -0.041140  0.000000 -0.000749   
         2014/02/28   TAM  0.507669 -0.004639  0.026279  0.000000 -0.154

In [8]:
# score.py
from __future__ import division
import numpy as np
import scipy.stats as ss
import pandas as pd
import scipy.sparse as _sparse
import multiprocessing as _multiprocessing
import logging

# it would take a lot of time to load the data again and again. so later find a way to customize list
# when making this change create a function to process the data

# when adding bootstrapping, think about not having to standardize again and again
class Score(object):
    # factors should have a column for equity and forward return

    def __init__(self, df, gamma, list_of_factors, tc=False):
        self.df = df

        self.gamma = gamma
        self.list = list_of_factors
        self.tc = tc

        # processing
        if self.tc:
            self.factors = self.df[['date', 'PERMNO', 'gvkey', 'equity', 'forward_r', 'fr'] + self.list]
        else:
            self.factors = self.df[['date', 'PERMNO', 'gvkey', 'equity', 'forward_r'] + self.list]
        self.factors = self.factors.replace([np.inf, -np.inf], 0)
        self.factors = self.factors.dropna()
        self.factors = self.factors[self.factors.equity != 0]
        self.factors = self.factors.drop_duplicates(subset=['date', 'gvkey'])
        self.factors = self.factors.set_index('date')
        index = pd.to_datetime(self.factors.index, format = '%Y/%m/%d')
        self.factors = self.factors.set_index(index.date)
        self.dates = sorted(self.factors.index.unique())

        self.x = np.array([ss.zscore(self.factors[self.factors.index == i][self.list].as_matrix(), axis=0) for i in self.dates])
        vv = [len(i) if not np.isnan(i).any() else 0 for i in self.x]
        index = [i for i,v in enumerate(vv) if v>100]
        self.dates = np.array(self.dates)[index]
        self.factors = self.factors[self.factors.index.isin(self.dates)]
        self.x = np.array([ss.zscore(self.factors[self.factors.index == i][self.list].as_matrix(), axis=0) for i in self.dates])
        self.r = np.array([np.array(self.factors[self.factors.index == i]['forward_r']) for i in self.dates])


# have a function for theta using a universal x and r
# the main theta value, save it intialization and print not return. instead should return a weights dataframe
# i dont want the weights dataframe when bootstrapping
# a single bootstrapping function. Alternatives to loops>? check bootstrapping packages source codes and maybe use multiprocessing

    def gettheta(self, csv_path = None):
        T = len(self.dates)
        K = len(self.list)
        # x and r are lists of length T
        # standardize each xt
        # x is nxk where N keeps changing
        weights = np.array(self.factors.pivot_table(values='equity', columns='gvkey', index=self.factors.index))
        w_dash = np.nan_to_num(weights)
        w_sum = np.array(w_dash.sum(axis=1))
        w_bar = weights / w_sum[:, np.newaxis]
        self.w_bar = w_bar
        r_bm = [np.dot(w_bar[i][~np.isnan(w_bar[i])], self.r[i]).T for i in range(T)]
        xrt = [np.dot(self.x[i].T, self.r[i]) / self.x[i].shape[0] for i in range(T)]
        mean_xrt = np.mean(xrt, axis=0)
        q1 = [xrt[i] - mean_xrt for i in range(T)]
        b = np.sum([np.dot(q1[i].reshape((K, 1)), q1[i].reshape((1, K))) for i in range(T)], axis=0) / T
        b_inv = np.linalg.inv(b)
        mean_rbm = np.mean(r_bm, axis=0)
        q2 = [r_bm[i] - mean_rbm for i in range(T)]
        b2 = np.sum([np.dot(q1[i], q2[i]) for i in range(T)], axis=0) / T
        c = np.dot(b_inv, b2)
        t1 = np.dot(b_inv, mean_xrt) / self.gamma
        theta = t1 - c
        self.theta = theta

        # output weights for further analysis
        # think about inheritance to analysis class
        permno = [self.factors[self.factors.index == i]['PERMNO'] for i in self.dates]
        gvkey = [self.factors[self.factors.index == i]['gvkey'] for i in self.dates]
        equity = [self.factors[self.factors.index == i]['equity'] for i in self.dates]
        if self.tc:
            orig_r = [self.factors[self.factors.index == i]['tc'] for i in self.dates]
        w_active = [np.dot(np.reshape(self.x[i], (self.x[i].shape[0],K)), theta) / self.x[i].shape[0] for i in range(T)]
        w_tot = [w_bar[i][~np.isnan(w_bar[i])] + w_active[i] for i in range(T)]
        r_tot = [np.multiply(w_tot[i], self.r[i]) for i in range(T)]
        r_s = [r_tot[i].sum() for i in range(T)]
        if self.tc:
            l = pd.DataFrame({'permno': pd.concat((i for i in permno)), 'gvkey': pd.concat((i for i in gvkey)),
                              'forward_r': np.hstack((i for i in self.r)),
                              'tc': pd.concat((i for i in orig_r)),
                              'w_bar': np.hstack((i for i in w_bar[~np.isnan(w_bar)])),
                              'w_active': np.hstack((i for i in w_active)), 'r_tot': np.hstack((i for i in r_tot)),
                              'w_tot': np.hstack((i for i in w_tot)), 'equity': pd.concat((i for i in equity))})
        else:
            l = pd.DataFrame({'permno': pd.concat((i for i in permno)), 'gvkey': pd.concat((i for i in gvkey)), 'forward_r': np.hstack((i for i in self.r)),
                          'w_bar': np.hstack((i for i in w_bar[~np.isnan(w_bar)])),
                          'w_active': np.hstack((i for i in w_active)), 'r_tot': np.hstack((i for i in r_tot)),
                          'w_tot': np.hstack((i for i in w_tot)), 'equity': pd.concat((i for i in equity))})

        print('Theta: {0}\nMean Monthly Return: {1}\n STD Monthly Return: {2}'.format(theta, np.mean(r_s), np.std(r_s)))

        if csv_path is not None:
            l.to_csv(csv_path)

        return l

    def resample(self):
        # number of dates
        date_num = np.size(self.dates)
        # pick resample dates
        sample_index = np.random.choice(date_num, date_num, replace=True)
        x = self.x[sample_index]
        r = self.r[sample_index]
        w_bar = self.w_bar[sample_index]
        return x, r, w_bar

    def _theta(self, x, r, w_bar):
        T = len(self.dates)
        K = len(self.list)
        r_bm = [np.dot(w_bar[i][~np.isnan(w_bar[i])], r[i]).T for i in range(T)]
        xrt = [np.dot(x[i].T, r[i]) / x[i].shape[0] for i in range(T)]
        mean_xrt = np.mean(xrt, axis=0)
        q1 = [xrt[i] - mean_xrt for i in range(T)]
        b = np.sum([np.dot(q1[i].reshape((K, 1)), q1[i].reshape((1, K))) for i in range(T)], axis=0) / T
        b_inv = np.linalg.inv(b)
        mean_rbm = np.mean(r_bm, axis=0)
        q2 = [r_bm[i] - mean_rbm for i in range(T)]
        b2 = np.sum([np.dot(q1[i], q2[i]) for i in range(T)], axis=0) / T
        c = np.dot(b_inv, b2)
        t1 = np.dot(b_inv, mean_xrt) / self.gamma
        theta = t1 - c
        return theta

    def bootstrapping(self, size=1000):
        results = []
        for i in range(size):
            a, r, w_bar = self.resample()
            results.append(self._theta(a,r,w_bar))
        return ss.sem(np.array(results, dtype=np.float), nan_policy='omit')


In [5]:
# winsorize

import pandas as pd
import numpy as np

'''
This file winsorizes the factors on an individual basis. 
i.e. we dont remove observations where earnings are negative

'''

'''
input - all_factors.csv or final_A.csv and final_q.csv
output - winsor_factors_univariate.csv
'''

dfa = pd.read_csv('final_A.csv')
dfq = pd.read_csv('final_Q.csv')

df = pd.merge(dfa, dfq)
df = df.dropna(subset=['equity', 'forward_r'])
df.to_csv('all_factors.csv', index=False)

df = pd.read_csv('all_factors.csv')

##
#remove max(10%, 10M) by equity and less than $1

def size_screen(df):
    df = df[df.equity > max(df.equity.quantile(0.1), 10)]
    df = df[df.PRC > 1]
    return df

df = df.groupby('date').apply(size_screen)

##
# drop -ve bm
df['b_m'] = np.where(df['b_m'] < 0, np.nan, df['b_m'])

# set max nei equal to 8
df['nei'] = np.where(df.nei>8, 8, df.nei)

# set -ve e_p to None
df['e_p'] = np.where(df['e_p'] < 0, np.nan, df['e_p'])

# set -ve cf_p to None
df['cf_p'] = np.where(df['cf_p'] < 0, np.nan, df['cf_p'])

# winsorize factors at 99
# we dont winsorize nei
dfo = df.copy()

factors = ['b_m', 'e_p', 'cf_p', 'o', 'no', 'i_a', 'noa', 'pia', 'ig', 'nsi', 'ivg', 'ivc', 'oa', 'poa', 'pta', 'gpa',
           'oc_a', 'ol', 'R6', 'R11', 'sue', 'roe', 'roa']

######
fb = ['o', 'no', 'noa', 'pia', 'ig', 'nsi', 'ivg', 'ivc', 'oa', 'poa', 'pta', 'gpa',
           'oc_a', 'ol', 'R6', 'R11', 'roe', 'roa']

fo = ['b_m', 'e_p', 'cf_p', 'sue', 'i_a']

df = df.replace([-np.inf, np.inf], np.nan)
dfo = dfo.replace([-np.inf, np.inf], np.nan)

def WinsorizeCustom(data, lower, upper):
    quantiles = data.quantile([lower, upper])
    q_l = quantiles.loc[lower]
    q_u = quantiles.loc[upper]

    out = np.where(data.values <= q_l, q_l, np.where(data >= q_u, q_u, data))
    return out

'''
for i in fb:
    df[i] = WinsorizeCustom(df[i], 0.01, 0.99)

for i in fo:
    df[i] = WinsorizeCustom(df[i], 0, 0.99)
    '''
# cross-sectional
for i in fb:
    df[i] = df.groupby('date')[i].transform(WinsorizeCustom, 0.01, 0.99)

for i in fo:
    df[i] = df.groupby('date')[i].transform(WinsorizeCustom, 0, 0.99)


df['forward_r'] = df.forward_r.replace(['C', 'B'], np.nan)
df['forward_r'] = df.forward_r.astype('float64')

def s(df):
    df['adj_SPREAD'] = df.SPREAD / df.CFACPR
    df['tr'] = df.adj_SPREAD / df.adj_price
    df.tr = df.tr.fillna(method='ffill')
    df.tr = df.tr.fillna(method='bfill')
    df.tr = df.tr.fillna(0)

    return df

df = df.groupby('gvkey').apply(s)

df['tr'] = WinsorizeCustom(df['tr'], 0.01, 0.99)

def s2(df):
    df['trc'] = (1 - df.tr.shift(-1)/10) / (1 + df.tr/10)
    return df

df = df.groupby('gvkey').apply(s2)



df.to_csv('winsor_factors_univariate.csv')



C:\Users\zchan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\zchan\Anaconda3\lib\site-packages\ipykernel_launcher.py:82: FutureWarning: 'date' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
C:\Users\zchan\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: RuntimeWarning: invalid value encountered in less_equal
C:\Users\zchan\Anaconda3\lib\site-packages\ipykernel_launcher.py:85: FutureWarning: 'date' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version


In [9]:
import pandas as pd
import numpy as np
#import Score
import portfolio

factors = ['b_m', 'e_p', 'cf_p', 'o', 'no', 'i_a', 'noa', 'pia', 'ig', 'nsi', 'ivg', 'ivc', 'oa', 'poa', 'pta', 'gpa',
           'oc_a', 'ol', 'R6', 'R11', 'sue', 'roe', 'roa', 'nei']
'''
Aversion = 3; optimal
'''
def scores(file, factors, gamma=3):

    df = pd.read_csv(file)

    theta = []
    se = []

    for i in factors:
        print(i)
        f = Score(df, gamma, [i])
        t = f.gettheta(i+'.csv')
        theta.append(f.theta[0])
        se.append(None)
        #se.append(f.bootstrapping(size=1000)[0])
    dfse = pd.DataFrame({'factors':factors, 'theta':theta, 'se':se})
    dfse.to_csv('univariate_theta.csv')

    return 'done'

scores('winsor_factors_univariate.csv', factors)


# portfolio
portfolio('univariate_theta.csv', 'univariate_portfolio.csv')

'''
aversion = 5; optimal
'''
## gamma = 5
def scores5(file, factors, gamma=5):

    df = pd.read_csv(file)

    theta = []
    se = []

    for i in factors:
        print(i)
        f = Score(df, gamma, [i])
        t = f.gettheta(i+'_5.csv')
        theta.append(f.theta[0])
        se.append(None)
        #se.append(f.bootstrapping(size=1000)[0])
    dfse = pd.DataFrame({'factors':factors, 'theta':theta, 'se':se})
    dfse.to_csv('univariate_theta_5.csv')

    return 'done'

scores5('winsor_factors_univariate.csv', factors)

portfolio('univariate_theta_5.csv', 'univariate_portfolio_5.csv', sub='_5')


'''
aversion = 3; long-only
'''
portfolio('univariate_theta.csv', 'univariate_portfolio_long_g3.csv', long_only=True)

'''
aversion = 5; long-only
'''
portfolio('univariate_theta_5.csv', 'univariate_portfolio_long_5.csv', sub='_5', long_only=True)


'''
transaction_cost 1/5
aversion = 5
'''

df = pd.read_csv('winsor_factors_univariate.csv')
df['fr'] = df.forward_r
df['forward_r'] = ((1+df.forward_r)*df.trc)-1


def scores5tc(file, factors, gamma=5, tc=True):

    df = pd.read_csv(file)

    theta = []
    se = []
    df['fr'] = df.forward_r
    df['forward_r'] = ((1 + df.forward_r) * df.trc) - 1

    for i in factors:
        print(i)
        f = Score(df, gamma, [i])
        t = f.gettheta(i+'_5_tc5.csv')
        theta.append(f.theta[0])
        se.append(f.bootstrapping(size=1000)[0])

    dfse = pd.DataFrame({'factors':factors, 'theta':theta, 'se':se})
    dfse.to_csv('univariate_theta_5_tc5.csv')

    return 'done'

scores5tc('winsor_factors_univariate.csv', factors)

portfolio('univariate_theta_5_tc5.csv', 'univariate_portfolio_5_tc5.csv', sepresent=True, sub='_5_tc5')

# by 3 i.e. 6
'''
transaction_cost 1/3
aversion = 5
'''
def scores3tc(file, factors, gamma=5, tc=True):

    df = pd.read_csv(file)

    def s2(df):
        df['trc'] = (1 - df.tr.shift(-1) / 6) / (1 + df.tr / 6)
        return df

    df = df.groupby('gvkey').apply(s2)

    theta = []
    se = []
    df['fr'] = df.forward_r
    df['forward_r'] = ((1 + df.forward_r) * df.trc) - 1

    for i in factors:
        print(i)
        f = Score(df, gamma, [i])
        t = f.gettheta(i+'_5_tc3.csv')
        theta.append(f.theta[0])
        se.append(None)

    dfse = pd.DataFrame({'factors':factors, 'theta':theta, 'se':se})
    dfse.to_csv('univariate_theta_5_tc3.csv')

    return 'done'

scores3tc('winsor_factors_univariate.csv', factors)

portfolio('univariate_theta_5_tc3.csv', 'univariate_portfolio_5_tc3.csv', sepresent=True, sub='_5_tc3')


# before and after

def publication_score(file, factors, gamma=5):

    df = pd.read_csv(file)
    lit = pd.read_csv('factors_Y.csv')
    df['date'] = pd.to_datetime(df.date, format='%Y/%m/%d')

    theta_b = []
    theta_a = []

    for i in factors:
        print(i)
        start = lit[lit.fname==i].Date.item()
        st = np.datetime64(start)
        dftb = df[df.date<start]
        dfta = df[df.date>=start]

        fb = Score(dftb, gamma, [i])
        tb = fb.gettheta(i+'_b.csv')
        theta_b.append(fb.theta[0])

        fa = Score(dfta, gamma, [i])
        ta = fa.gettheta(i + '_a.csv')
        theta_a.append(fa.theta[0])


    dfse = pd.DataFrame({'factors':factors, 'theta_before':theta_b, 'theta_after':theta_a})
    dfse.to_csv('publication_theta.csv')

    return 'done'

publication_score('winsor_factors_univariate.csv', factors)

portfolio('publication_theta_b.csv', 'univariate_portfolio_5_b.csv', sub='_b')

portfolio('publication_theta_a.csv', 'univariate_portfolio_5_a.csv', sub='_a')

## NBER recession
'''
'1969-12-01' - '1970-11-30'
'1972-11-01' - '1975-03-31'
'1980-01-01' - '1980-07-31'
'1981-07-01' - '1982-11-30'
'1990-07-01' - '1991-03-31'
'2001-03-01' - '2001-11-30'
'2007-12-01' - '2009-06-30'

note: the returns are for the forward month
'''
def recession_score(file, factors, gamma=5):

    df = pd.read_csv(file)
    df=df.append(pd.Series(name='2018-12-31'))
    df = df.shift(1)
    #df['date'] = pd.to_datetime(df.date, format='%Y/%m/%d')

    condlist = [((df.date>='1969-12-01') & (df.date<= '1970-11-30')) |
                ((df.date>='1972-11-01') & (df.date<= '1975-03-31')) |
                ((df.date>='1980-01-01') & (df.date<= '1980-07-31')) |
                ((df.date>='1981-07-01') & (df.date<= '1982-11-30')) |
                ((df.date>='1990-07-01') & (df.date<= '1991-03-31')) |
                ((df.date>='2001-03-01') & (df.date<= '2001-11-30')) |
                ((df.date >= '2007-12-01') & (df.date <= '2009-06-30')), True]
    choicelist = ['R', 'N']
    df['rec'] = np.select(condlist, choicelist)

    dfr = df[df.rec=='R']
    dfn = df[df.rec=='N']

    theta_r = []
    theta_n = []

    for i in factors:

        fr = Score(dfr, gamma, [i])
        tr = fr.gettheta(i+'_r.csv')
        theta_r.append(fr.theta[0])

        fn = Score(dfn, gamma, [i])
        tn = fn.gettheta(i + '_n.csv')
        theta_n.append(fn.theta[0])


    dfse = pd.DataFrame({'factors':factors, 'theta_r':theta_r, 'theta_n':theta_n})
    dfse.to_csv('recession_theta.csv')

    return 'done'

recession_score('winsor_factors_univariate.csv', factors)

portfolio('recession_theta_n.csv', 'univariate_portfolio_5_n.csv', sub='_n')

portfolio('recession_theta_r.csv', 'univariate_portfolio_5_r.csv', sub='_r')

'''
second and better approach
refer to scorev2 regarding the changes required in statsmodels package
'''
# from scorev2 import Score

'''
recession cyclicilty
'''
def recession_scorev2(file, factors, gamma=5):

    df = pd.read_csv(file)
    df=df.append(pd.Series(name='2018-12-31'))
    df = df.shift(1)
    #df['date'] = pd.to_datetime(df.date, format='%Y/%m/%d')

    condlist = [((df.date>='1969-12-01') & (df.date<= '1970-11-30')) |
                ((df.date>='1972-11-01') & (df.date<= '1975-03-31')) |
                ((df.date>='1980-01-01') & (df.date<= '1980-07-31')) |
                ((df.date>='1981-07-01') & (df.date<= '1982-11-30')) |
                ((df.date>='1990-07-01') & (df.date<= '1991-03-31')) |
                ((df.date>='2001-03-01') & (df.date<= '2001-11-30')) |
                ((df.date >= '2007-12-01') & (df.date <= '2009-06-30')), True]
    choicelist = ['R', 'N']
    df['rec'] = np.select(condlist, choicelist)

    theta_r = []
    theta_n = []

    for i in factors:
        df[i+'r'] = df[i] * np.where(df.rec == 'R', 1, 0)
        df[i + 'n'] = df[i] * np.where(df.rec == 'N', 1, 0)

        fr = Score(df, gamma, [i+'r', i+'n'])
        tr = fr.gettheta(i+'_c.csv')
        theta_r.append(fr.theta[0])
        theta_n.append(fr.theta[1])



    dfse = pd.DataFrame({'factors':factors, 'theta_r':theta_r, 'theta_n':theta_n})
    dfse.to_csv('recession_theta.csv')

    return 'done'

recession_scorev2('winsor_factors_univariate.csv', factors)

#portfolio on breaktest.py

# todo: possible error
# from scorev2 import Score
def publication_scorev2(file, factors, gamma=5):

    df = pd.read_csv(file)
    lit = pd.read_csv('factors_Y.csv')
    df = df.append(pd.Series(name='2018-12-31'))
    df = df.shift(1)
    #df['date'] = pd.to_datetime(df.date, format='%Y/%m/%d')

    theta_b = []
    theta_a = []

    for i in factors:
        print(i)
        start = lit[lit.fname==i].Date.item()
        df['D'] = np.where(df.date < start, 'B', 'A')
        df[i+'b'] = df[i] * np.where(df.D == 'B', 1, 0)
        df[i + 'a'] = df[i] * np.where(df.D == 'A', 1, 0)

        fb = Score(df, gamma, [i+'b', i+'a'])
        tb = fb.gettheta(i+'_p.csv')
        theta_b.append(fb.theta[0])
        theta_a.append(fb.theta[1])

    dfse = pd.DataFrame({'factors':factors, 'theta_before':theta_b, 'theta_after':theta_a})
    dfse.to_csv('publication_theta.csv')

    return 'done'

publication_scorev2('winsor_factors_univariate.csv', factors)

C:\Users\zchan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


b_m


C:\Users\zchan\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\zchan\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Theta: [4.33814517]
Mean Monthly Return: 0.02309741734976739
 STD Monthly Return: 0.079422420177775
e_p
Theta: [5.01414503]
Mean Monthly Return: 0.022008312270214166
 STD Monthly Return: 0.07694281658247287
cf_p
Theta: [4.56982577]
Mean Monthly Return: 0.020612399810014264
 STD Monthly Return: 0.07300840467317404
o
Theta: [4.09880344]
Mean Monthly Return: 0.013713152746208709
 STD Monthly Return: 0.050246663397840514
no
Theta: [4.44137232]
Mean Monthly Return: 0.015193380838167186
 STD Monthly Return: 0.05426239492998209
i_a
Theta: [-7.28748625]
Mean Monthly Return: 0.02591304631689649
 STD Monthly Return: 0.07694042150064892
noa
Theta: [-8.73165231]
Mean Monthly Return: 0.034284143768014634
 STD Monthly Return: 0.10193771958582824
pia
Theta: [-14.48043171]
Mean Monthly Return: 0.051672879258505236
 STD Monthly Return: 0.11855182748223618
ig
Theta: [-6.43822639]
Mean Monthly Return: 0.017408303650311405
 STD Monthly Return: 0.06200601064299096
nsi
Theta: [-7.37065874]
Mean Monthly Retu

C:\Users\zchan\Anaconda3\lib\site-packages\scipy\stats\stats.py:2253: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


Theta: [2.36417744]
Mean Monthly Return: 0.015323521604680559
 STD Monthly Return: 0.06278459134154696
R11
Theta: [2.08003114]
Mean Monthly Return: 0.014551063784557334
 STD Monthly Return: 0.06224233946118529
sue
Theta: [10.74298962]
Mean Monthly Return: 0.04500892444717047
 STD Monthly Return: 0.11980099794059444
roe
Theta: [3.85128727]
Mean Monthly Return: 0.017621183288659033
 STD Monthly Return: 0.06804379964809715
roa
Theta: [2.50878936]
Mean Monthly Return: 0.015815545937566462
 STD Monthly Return: 0.06273152430435304
nei
Theta: [5.01283563]
Mean Monthly Return: 0.01864792997726091
 STD Monthly Return: 0.07551911283283977


TypeError: 'module' object is not callable